<a href="https://colab.research.google.com/github/earo12/Deploy_Consultancy/blob/dev-branch/Automatizacion_consultora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [4]:
# Conexión a la BD:

def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='admin',
            database='consultora_prueba'
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print(f"Error al conectar con MySQL: {e}")
        return None


In [6]:
# Función para crear un INSERT o un UPDATE desde un csv

def insert_or_update_data(df, connection):
  cursor = connection.cursor()
  query = """
              INSERT INTO datos_consultora (
                id_consultor, apellidos, nombre, correo_electronico, telefono, estado_actual, fecha_inicio_estado_actual,
                duracion_estado_actual, nombre_proyecto, rol_proyecto, fecha_inicio_asignacion, fecha_fin_asignacion,
                duracion_dias_proyecto, fecha_ingreso_pool, comentarios_disponibildad, ultima_actualizacion_estado, responsable_seguimiento,
                habilidades_principales, idiomas, ubicacion_actual
              )

              VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
              )

              ON DUPLICATE KEY UPDATE
              estado_actual = VALUES(estado_actual),
              fecha_inicio_estado_actual = VALUES(fecha_inicio_estado_actual),
              duracion_estado_actual = VALUES(duracion_estado_actual),
              nombre_proyecto = VALUES(nombre_proyecto),
              fecha_inicio_asignacion = VALUES(fecha_inicio_asignacion),
              fecha_fin_asignacion = VALUES(fecha_fin_asignacion),
              duracion_dias_proyecto = VALUES(duracion_dias_proyecto),
              comentarios_disponibildad = VALUES(comentarios_disponibildad),
              ultima_actualizacion_estado = VALUES(ultima_actualizacion_estado),
          """
  for _, row in df.iterrows():
      values = tuple(row)
      cursor.execute(query, values)
  connection.commit()
  cursor.close()
print("Tabla actualizada")

Tabla actualizada


In [8]:
# Cargar datos desde el csv

def load_data_csv(file_path):
  try:
    df = pd.read_csv(file_path)
    return df
  except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    return None

In [9]:
# Flujo de datos:

def main():
  connection = connect_to_db()
  if connection:
    file_path = 'datos_consultora.csv'
    df = load_data_csv(file_path)
    if df is not None:
      insert_or_update_data(df, connection)
      connection.close()
      print("Conexión cerrada")